In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.layers import Dense, Embedding, LSTM, GRU, SimpleRNN, Dropout
from sklearn.model_selection import train_test_split

In [2]:
pretrained_model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(pretrained_model, input_shape=[], dtype=tf.string, trainable=True)

In [3]:
embed = hub.load("https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1")
embeddings = embed(["cat is on the mat dog is in the fog"])

embeddings

<tf.Tensor: shape=(1, 20), dtype=float32, numpy=
array([[ 1.2917835 ,  0.6960632 ,  0.04837722,  0.7461801 , -0.20820257,
        -0.12027001, -0.7988958 ,  0.84001   , -0.02885479, -0.83123875,
        -1.4375954 ,  1.1317196 , -0.5531615 , -0.45697683, -0.9780804 ,
         1.0363967 , -0.2558647 ,  0.24833947, -1.6134161 , -0.55305725]],
      dtype=float32)>

In [4]:
path = "../Clean/lemma_allresult.csv"
dataset = pd.read_csv(path)

In [5]:
feat = dataset.iloc[:,0]
target = dataset.iloc[:,1]

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(feat,target, test_size = 0.2, train_size = 0.8, random_state = 42)

In [7]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Reshape((20, 1), input_shape=(20,)))
model.add(tf.keras.layers.SimpleRNN(32))
model.add(tf.keras.layers.Dense(16,activation="relu"))
model.add(Dropout(0.5))
model.add(tf.keras.layers.Dense(1,activation="sigmoid"))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
reshape (Reshape)            (None, 20, 1)             0         
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 32)                1088      
_________________________________________________________________
dense (Dense)                (None, 16)                528       
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 401,653
Trainable params: 401,653
Non-trainable params: 0
__________________________________________________

In [8]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [9]:
model.fit(X_train,
          Y_train,
         epochs=10,
         validation_split=0.2,
         verbose=1)

Train on 6400 samples, validate on 1600 samples
Epoch 1/10
6400/6400 [==============================] - 6s 963us/sample - loss: 0.6230 - accuracy: 0.6491 - val_loss: 0.5431 - val_accuracy: 0.7269
Epoch 2/10
6400/6400 [==============================] - 4s 633us/sample - loss: 0.5130 - accuracy: 0.7539 - val_loss: 0.4565 - val_accuracy: 0.7881
Epoch 3/10
6400/6400 [==============================] - 4s 621us/sample - loss: 0.4188 - accuracy: 0.8233 - val_loss: 0.4215 - val_accuracy: 0.7994
Epoch 4/10
6400/6400 [==============================] - 4s 630us/sample - loss: 0.3311 - accuracy: 0.8736 - val_loss: 0.3915 - val_accuracy: 0.8256
Epoch 5/10
6400/6400 [==============================] - 4s 634us/sample - loss: 0.2637 - accuracy: 0.9073 - val_loss: 0.3612 - val_accuracy: 0.8450
Epoch 6/10
6400/6400 [==============================] - 4s 618us/sample - loss: 0.2174 - accuracy: 0.9294 - val_loss: 0.3929 - val_accuracy: 0.8444
Epoch 7/10
6400/6400 [==============================] - 4s 614us

In [10]:
score,acc = model.evaluate(X_test, Y_test, verbose = 1, batch_size = 32)

2000/2000 [==============================] - 1s 374us/sample - loss: 0.5492 - accuracy: 0.8430


In [11]:
np.set_printoptions(suppress=True)
result = model.predict(X_test)

result

array([[0.00210623],
       [0.01230465],
       [0.00066478],
       ...,
       [0.99908733],
       [0.00082071],
       [0.8957933 ]], dtype=float32)

In [12]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true=Y_test , y_pred=result>0.5)

cm

array([[837, 159],
       [155, 849]], dtype=int64)